Carga el conjunto de datos MNIST y divídelo en un conjunto de entrenamiento, un conjunto de validación y un conjunto de prueba (por ejemplo, utiliza 50,000 instancias para el entrenamiento, 10,000 para la validación y 10,000 para las pruebas). 

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from utiles import plot_decision_regions

In [2]:
from sklearn.datasets import fetch_openml

X_mnist, y_mnist = fetch_openml('mnist_784', return_X_y=True, as_frame=False, parser='auto')

In [3]:
X_train, y_train = X_mnist[:50000], y_mnist[:50000]
X_valid, y_valid = X_mnist[50000:60000], y_mnist[50000:60000]
X_test, y_test = X_mnist[60000:], y_mnist[60000:]

Entrena clasificadores individuales, por ejemplo un clasificador RandomForest, un clasificador SVM y un MLP. 

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
rnd_clf = RandomForestClassifier(n_estimators=500,
                                 bootstrap=True, 
                                 max_leaf_nodes=16,
                                 n_jobs=-1, 
                                 max_features='sqrt',
                                 oob_score=True,
                                 random_state=0)
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       oob_score=True, random_state=0)

In [6]:
rnd_clf.oob_score_

0.81824

In [7]:
maq_sv = SVC(kernel='rbf', C=10, gamma=0.01, random_state=0, tol = 4)
maq_sv.fit(X_train, y_train)

SVC(C=10, gamma=0.01, random_state=0, tol=4)

In [8]:
maq_sv.score(X_valid, y_valid)

0.0961

In [11]:
from sklearn.neural_network import MLPClassifier
MLP_cl = MLPClassifier(hidden_layer_sizes=(100,), alpha=0.0001, solver='adam', verbose=10,  random_state=0, tol=0.01)
MLP_cl.fit(X_train, y_train)

Iteration 1, loss = 3.54431608
Iteration 2, loss = 0.94013219
Iteration 3, loss = 0.51327037
Iteration 4, loss = 0.36260604
Iteration 5, loss = 0.27181572
Iteration 6, loss = 0.22013834
Iteration 7, loss = 0.18347398
Iteration 8, loss = 0.16055194
Iteration 9, loss = 0.14253513
Iteration 10, loss = 0.12549304
Iteration 11, loss = 0.12593825
Iteration 12, loss = 0.12069661
Iteration 13, loss = 0.10509758
Iteration 14, loss = 0.10898608
Iteration 15, loss = 0.10087105
Iteration 16, loss = 0.09609829
Iteration 17, loss = 0.10376947
Iteration 18, loss = 0.10125230
Iteration 19, loss = 0.09667420
Iteration 20, loss = 0.10051940
Iteration 21, loss = 0.08603826
Iteration 22, loss = 0.08491675
Iteration 23, loss = 0.09274767
Iteration 24, loss = 0.07592097
Iteration 25, loss = 0.07730937
Iteration 26, loss = 0.07394752
Iteration 27, loss = 0.07524461
Iteration 28, loss = 0.07970502
Iteration 29, loss = 0.06969892
Iteration 30, loss = 0.05806532
Iteration 31, loss = 0.06642944
Iteration 32, los

MLPClassifier(random_state=0, tol=0.01, verbose=10)

In [12]:
MLP_cl.score(X_valid, y_valid)

0.9608

A continuación, intenta combinarlos en un ensamble cuyo desempeño supere a cada clasificador individual en el conjunto de validación, utilizando votación por mayoría hard y soft. Una vez que hayas encontrado uno, pruébalo en el conjunto de prueba.

In [17]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf),
                ('smv', maq_sv),
                ('mlp', MLP_cl)]
)
voting_clf.fit(X_train, y_train)

Iteration 1, loss = 3.54431608
Iteration 2, loss = 0.94013219
Iteration 3, loss = 0.51327037
Iteration 4, loss = 0.36260604
Iteration 5, loss = 0.27181572
Iteration 6, loss = 0.22013834
Iteration 7, loss = 0.18347398
Iteration 8, loss = 0.16055194
Iteration 9, loss = 0.14253513
Iteration 10, loss = 0.12549304
Iteration 11, loss = 0.12593825
Iteration 12, loss = 0.12069661
Iteration 13, loss = 0.10509758
Iteration 14, loss = 0.10898608
Iteration 15, loss = 0.10087105
Iteration 16, loss = 0.09609829
Iteration 17, loss = 0.10376947
Iteration 18, loss = 0.10125230
Iteration 19, loss = 0.09667420
Iteration 20, loss = 0.10051940
Iteration 21, loss = 0.08603826
Iteration 22, loss = 0.08491675
Iteration 23, loss = 0.09274767
Iteration 24, loss = 0.07592097
Iteration 25, loss = 0.07730937
Iteration 26, loss = 0.07394752
Iteration 27, loss = 0.07524461
Iteration 28, loss = 0.07970502
Iteration 29, loss = 0.06969892
Iteration 30, loss = 0.05806532
Iteration 31, loss = 0.06642944
Iteration 32, los

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_leaf_nodes=16,
                                                     n_estimators=500,
                                                     n_jobs=-1, oob_score=True,
                                                     random_state=0)),
                             ('smv',
                              SVC(C=10, gamma=0.01, random_state=0, tol=4)),
                             ('mlp',
                              MLPClassifier(random_state=0, tol=0.01,
                                            verbose=10))])

In [18]:
voting_clf.score(X_valid, y_valid)

0.8704

In [20]:
voting_clf.voting = "soft"
voting_clf.named_estimators["smv"].probability = True
voting_clf.fit(X_train, y_train)

Iteration 1, loss = 3.54431608
Iteration 2, loss = 0.94013219
Iteration 3, loss = 0.51327037
Iteration 4, loss = 0.36260604
Iteration 5, loss = 0.27181572
Iteration 6, loss = 0.22013834
Iteration 7, loss = 0.18347398
Iteration 8, loss = 0.16055194
Iteration 9, loss = 0.14253513
Iteration 10, loss = 0.12549304
Iteration 11, loss = 0.12593825
Iteration 12, loss = 0.12069661
Iteration 13, loss = 0.10509758
Iteration 14, loss = 0.10898608
Iteration 15, loss = 0.10087105
Iteration 16, loss = 0.09609829
Iteration 17, loss = 0.10376947
Iteration 18, loss = 0.10125230
Iteration 19, loss = 0.09667420
Iteration 20, loss = 0.10051940
Iteration 21, loss = 0.08603826
Iteration 22, loss = 0.08491675
Iteration 23, loss = 0.09274767
Iteration 24, loss = 0.07592097
Iteration 25, loss = 0.07730937
Iteration 26, loss = 0.07394752
Iteration 27, loss = 0.07524461
Iteration 28, loss = 0.07970502
Iteration 29, loss = 0.06969892
Iteration 30, loss = 0.05806532
Iteration 31, loss = 0.06642944
Iteration 32, los

0.9593

In [21]:
voting_clf.score(X_test, y_test)

0.9593

Prueba otros ensambles usando Bagging, Boosting y Stacking (Stacking utiliza validación cruzada por lo que es mejor dividir los datos sólo en entrenamiento y prueba)

In [22]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(maq_sv, 
                            n_estimators=100,
                            learning_rate=1, 
                            random_state=0,
                            algorithm='SAMME.R'
                             )
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=SVC(C=10, gamma=0.01, probability=True,
                                 random_state=0, tol=4),
                   learning_rate=1, n_estimators=100, random_state=0)

In [ ]:
plt.figure(figsize=(8,6))
plot_decision_regions(ada_clf, X_train, y_train)
plt.xlim(-1.5,2.5)
plt.ylim(-1.5,2)
plt.show()

print(ada_clf.score(X_test, y_test))